In [7]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import sys
import os
import pickle
from datetime import datetime

sys.path.append("../")
from cleanair.models import ModelData
from cleanair import metrics
from cleanair.dashboard.components import ModelFitComponent

In [8]:
# setup your filepaths
secrets_dir = "../.secrets/"
data_dir = "../data/"
results_dir = "../data/"
secretfile=os.path.join(secrets_dir, "db_secrets.json")

# open the parser config
with open(os.path.join(secrets_dir, "config.json"), "r") as filepath:
    parser_config = json.load(filepath)

if parser_config["local_read"]:
    model_data = ModelData(config_dir=data_dir, secretfile=secretfile)
    
if parser_config["predict_read_local"]:
    y_test_pred_fp = os.path.join(results_dir, "test_pred.pickle")
    with open(y_test_pred_fp, "rb") as handle:
        y_test_pred = pickle.load(handle)
    model_data.update_test_df_with_preds(y_test_pred, datetime.now())
print(model_data.normalised_training_data_df.sample(3))

2020-02-26 18:10:29     INFO: Database connection information loaded from <_io.TextIOWrapper name='../.secrets/db_secrets.json' mode='r' encoding='UTF-8'>


                                  point_id measurement_start_utc source  \
4487  4041fd3c-a1c1-47ca-ab3e-d7501d34ba0f   2020-01-28 23:00:00    aqe   
2548  b48e3a23-a9a5-44f8-b2e2-6c8fc6d4edf7   2020-01-28 04:00:00   laqn   
1207  3d444b6d-f821-4830-aa10-9887958175ce   2020-01-28 07:00:00   laqn   

                                               location  in_london       lon  \
4487  0101000020e610000024f1f274ae28dabfb94f8e0244b9...       True -0.408733   
2548  0101000020e610000038f581e49d43dcbf2d78d15790be...       True -0.441627   
1207  0101000020e6100000f8214a940a5db6bfc9513cba38c1...       True -0.087357   

            lat          date_opened  date_closed  \
4487  51.447388  2009-08-19 00:00:00          NaN   
2548  51.488780  2004-01-01 00:00:00          NaN   
1207  51.509544  2007-04-22 00:00:00          NaN   

      value_1000_total_a_road_length  ...  value_10_total_b_road_length  \
4487                     4868.745559  ...                     18.178893   
2548           

In [9]:
# evaluate the metrics
metric_methods = metrics.get_metric_methods()
precision_methods = metrics.get_precision_methods(
    pe1=metrics.probable_error
)
sensor_scores_df, temporal_scores_df = metrics.evaluate_model_data(
    model_data,
    metric_methods,
    precision_methods=precision_methods,
)


training_set False

testing_set True


In [13]:
# wrap all the results up into an object to be visualised
model_fit_component = ModelFitComponent(0, model_data, sensor_scores_df, temporal_scores_df)

metric="mae"
pollutant="NO2"

# ToDo: show the interest points map



In [14]:
fig_dict = model_fit_component.get_temporal_metrics_timeseries(metric, pollutant)
fig = go.Figure(fig_dict)
fig.show()